# 18_Global_ValidationPipe_Options_whitelist_forbidNonWhitelisted_transform

In the previous lecture, we added **`ValidationPipe` globally** so that all incoming requests are validated automatically. Now we’ll go deeper into **configuration options** for `ValidationPipe` and how they affect request data.

---

## 1. Recap – Global ValidationPipe

In `main.ts` we registered a global `ValidationPipe`:

```ts
import { NestFactory } from '@nestjs/core';
import { AppModule } from './app.module';
import { ValidationPipe } from '@nestjs/common';

async function bootstrap() {
  const app = await NestFactory.create(AppModule);

  app.useGlobalPipes(new ValidationPipe({
    whitelist: true,
    forbidNonWhitelisted: true,
    transform: true,
  }));

  await app.listen(process.env.PORT ?? 3000);
}

bootstrap();
```

**Key idea:**

* Global `ValidationPipe` runs on **every request** handled by every controller.
* If a route parameter/body/query has a **DTO type**, NestJS uses that DTO + `class-validator` to validate.

---

## 2. The Request Flow With DTO + ValidationPipe

Example controller method:

```ts
import { Controller, Post, Body } from '@nestjs/common';
import { CreeateUserDto } from './dtos/create-user.dto';

@Controller('users')
export class UsersController {
  @Post()
  createUser(@Body() user: CreeateUserDto) {
    console.log(user);
    console.log(typeof user);
    console.log(user instanceof CreeateUserDto);

    return 'User created successfully';
  }
}
```

Example DTO:

```ts
import {
  IsBoolean,
  IsEmail,
  IsNotEmpty,
  IsNumber,
  IsOptional,
  IsString,
  MinLength,
} from 'class-validator';

export class CreeateUserDto {
  @IsNumber()
  id: number;

  @IsString({ message: 'Name should be a string value' })
  @IsNotEmpty()
  @MinLength(3, { message: 'Name is too short. Minimum length is 3 characters' })
  name: string;

  @IsString()
  @IsOptional()
  gender: string;

  @IsEmail()
  email: string;

  @IsBoolean()
  isMarried: boolean;
}
```

**What happens on a POST /users request?**

1. Request body is read by `@Body()`.
2. `ValidationPipe` runs globally and:

   * Validates the body **against** `CreeateUserDto` decorators.
   * Applies extra options: `whitelist`, `forbidNonWhitelisted`, `transform`.
3. If validation passes → controller method runs.
4. If validation fails → NestJS returns a **400 Bad Request** with details.

---

## 3. Problem: Extra / Unwanted Properties in Request Body

Imagine a client sends this JSON:

```json
{
  "id": 10,
  "name": "John Doe",
  "email": "john@example.com",
  "gender": "male",
  "isMarried": false,
  "age": 30
}
```

But in `CreeateUserDto` we **do not** have `age`.

Without any special options:

* The request **succeeds**.
* `user` object in the controller contains `age` as well.
* No error is thrown.

This is **dangerous** because:

* Attackers can send **extra / malicious data**.
* Your backend might accidentally trust or store data that **is not defined** in your DTO.

To solve this, we use **`whitelist`** and **`forbidNonWhitelisted`**.

---

## 4. `whitelist: true`

```ts
app.useGlobalPipes(new ValidationPipe({
  whitelist: true,
}));
```

**What it does:**

* Only keeps properties that are **decorated with validation decorators** (`@IsString`, `@IsEmail`, etc.) in the DTO.
* All other properties are **stripped out** before reaching the controller.

Using the same request:

```json
{
  "id": 10,
  "name": "John Doe",
  "email": "john@example.com",
  "gender": "male",
  "isMarried": false,
  "age": 30
}
```

Assuming all DTO properties are decorated:

* Incoming object has `age`.
* Inside the controller, `user` will be:

```ts
{
  id: 10,
  name: 'John Doe',
  email: 'john@example.com',
  gender: 'male',
  isMarried: false,
}
```

`age` is **silently removed**, not passed to your business logic.

> ⚠️ Note: Initially, when some properties (like `id` or `isMarried`) had **no decorators**, they were also dropped. After adding `@IsNumber()` and `@IsBoolean()` decorators, they were preserved.

---

## 5. `forbidNonWhitelisted: true`

```ts
app.useGlobalPipes(new ValidationPipe({
  whitelist: true,
  forbidNonWhitelisted: true,
}));
```

**What it does:**

* Instead of silently removing extra fields, NestJS will **throw an error** if any non-whitelisted property is present.

With this option:

* Sending `age` (which is **not** in `CreeateUserDto`) results in a **400 Bad Request** with a message like:

```json
{
  "statusCode": 400,
  "message": [
    "property age should not exist"
  ],
  "error": "Bad Request"
}
```

And importantly:

* The request **never reaches** the controller.
* No `console.log(user)` is executed.

This is great for **strict APIs**, where you want clients to send **only the fields you explicitly accept**.

---

## 6. `transform: true`

Initially, even with DTO typing:

```ts
createUser(@Body() user: CreeateUserDto) {
  console.log(typeof user);            // 'object'
  console.log(user instanceof CreeateUserDto); // false (before transform)
}
```

Even though TypeScript thinks `user` is `CreeateUserDto`, at runtime it is just a **plain JavaScript object**, not an **instance** of the class.

By enabling `transform: true`:

```ts
app.useGlobalPipes(new ValidationPipe({
  whitelist: true,
  forbidNonWhitelisted: true,
  transform: true,
}));
```

Now:

```ts
createUser(@Body() user: CreeateUserDto) {
  console.log(typeof user);            // 'object'
  console.log(user instanceof CreeateUserDto); // true ✅
}
```

**What `transform: true` does:**

* Uses **class-transformer** internally to convert plain JSON into a real **class instance** of your DTO.
* This allows you to:

  * Rely on `instanceof` checks.
  * Use class methods (if you define any on the DTO).
  * Get proper type conversion/behavior in more advanced scenarios.

> This is why we had to install **`class-transformer`**: so NestJS can actually transform the plain body into an instance of `CreeateUserDto`.

---

## 7. Summary of All Three Options

**Final global configuration in `main.ts`:**

```ts
app.useGlobalPipes(new ValidationPipe({
  whitelist: true,            // Strip unknown properties
  forbidNonWhitelisted: true, // Throw error if unknown properties are sent
  transform: true,            // Convert payloads to DTO instances
}));
```

### ✅ `whitelist: true`

* Removes all properties **not decorated** in DTOs.
* Protects from unwanted data leaking into your controller/service.

### ✅ `forbidNonWhitelisted: true`

* Makes your API **strict**.
* Rejects requests that contain extra/unknown fields.
* Great for security and clear client contracts.

### ✅ `transform: true`

* Automatically converts request bodies into **actual DTO instances**.
* Allows `instanceof` checks and usage of DTO methods.

---

## 8. Practical Takeaways

* Always use a DTO + validation decorators (`class-validator`) for **every input** you care about.
* Use **global `ValidationPipe`** so all routes are protected.
* Turn on:

  * `whitelist` → clean data
  * `forbidNonWhitelisted` → strict & safe
  * `transform` → real DTO instances

Together, these make your NestJS API:

* **Safer** (rejects malicious/extra fields)
* **Cleaner** (only expected fields flow through)
* **More predictable** (runtime instances match your DTO types)

This builds a solid foundation for handling **request validation** in any real-world NestJS application.

---
